In [1]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import geopandas as gpd
import numpy as np

nyc_zip = gpd.read_file("ZIP_CODE_040114.shp")
nyc_zip = nyc_zip.to_crs(epsg=4326)
nyc_zip = nyc_zip[['ZIPCODE',"POPULATION"]]
nyc_zip['ZIPCODE'] = nyc_zip['ZIPCODE'].astype('int')
nyc_zip = nyc_zip.reset_index(drop=True)

In [2]:
cwd = os.getcwd()

folder_path = cwd

df_list = [nyc_zip]

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.upper()
        if 'UNNAMED: 0' in df.columns:
            df = df.drop('UNNAMED: 0', axis=1)
        if 'GEOMETRY' in df.columns:
            df = df.drop('GEOMETRY', axis=1)
        df_list.append(df)
        
        #df.replace([np.inf, -np.inf], 0, inplace=True)
        
        df = df.replace([np.inf, -np.inf], np.nan) # replace inf values with NaN
        df = df.dropna()
        df['ZIPCODE'] = df['ZIPCODE'].astype('int')
        
merged_df = pd.concat(df_list, ignore_index=True).groupby('ZIPCODE').sum()
merged_df = merged_df[merged_df['POPULATION'] > 100]



merged_df.head()


,POPULATION,NORMALIZED EDP CALLS,NORMALIZED_TREE_COUNT,LIQUOR_SHOPS_NORMALIZED,NORMALIZED_SMOKE,KW_POTENTIAL,NORMALIZED_RESTAURANT_COUNT,NORMALIZED_SIDEWALK_WIDTH,SIDEWALK CONDITION COMPLAINTS,STREET LIGHT CONDITION COMPLAINTS,NORMALIZED_HEIGHT
ZIPCODE,,,,,,,,,,,
10001,44826.0,-0.072028,4063.912271,1.032275,-3.891336,17910.760607,0.973474,3.997069,-0.231414,-0.457780,106.780181
10002,162610.0,-2.378207,2784.949019,-0.121014,-7.034142,28102.041384,0.043964,2.578797,-0.196298,-0.248578,63.413626
10003,111756.0,-1.956528,3623.216752,0.176924,-6.238549,18187.268225,0.335342,5.127329,-0.377546,-0.413917,70.415532
10004,17496.0,-6.092033,1347.837480,9.281730,-2.335647,4689.146343,12.923743,0.804002,5.087163,3.976968,382.393478
10005,16214.0,-2.084577,1775.255437,-0.490690,-5.242604,2421.583870,0.076792,4.467777,1.276625,0.108954,334.487916


In [3]:
merged_df['POPULATION'].sum()

16796958.0

In [4]:
merged_df.shape

(183, 11)

In [5]:
population = merged_df['POPULATION']
merged_df.drop('POPULATION',axis=1,inplace=True)
cols = merged_df.columns
index = merged_df.index
scaler = StandardScaler()
merged_scaled_df = pd.DataFrame(scaler.fit_transform(merged_df),columns = cols,index=index)

In [6]:
merged_scaled_df['POPULATION'] = population

In [7]:
merged_scaled_df.to_csv('merged_df.csv')

In [8]:
merged_scaled_df

,NORMALIZED EDP CALLS,NORMALIZED_TREE_COUNT,LIQUOR_SHOPS_NORMALIZED,NORMALIZED_SMOKE,KW_POTENTIAL,NORMALIZED_RESTAURANT_COUNT,NORMALIZED_SIDEWALK_WIDTH,SIDEWALK CONDITION COMPLAINTS,STREET LIGHT CONDITION COMPLAINTS,NORMALIZED_HEIGHT,POPULATION
ZIPCODE,,,,,,,,,,,
10001,1.594939,-0.693092,1.030044,1.615201,-0.739393,0.949560,0.869223,-0.231860,-0.457869,0.871854,44826.0
10002,0.525973,-0.857512,-0.122022,0.009141,-0.458616,0.037679,-0.163255,-0.196765,-0.249181,0.185881,162610.0
10003,0.721431,-0.749747,0.175600,0.415711,-0.731775,0.323531,1.692033,-0.377904,-0.414114,0.296637,111756.0
10004,-1.195468,-1.042264,9.270756,2.410201,-1.103657,12.673172,-1.455277,5.083505,3.965994,5.231510,17496.0
10005,0.662077,-0.987316,-0.491307,0.924666,-1.166130,0.069885,1.211890,1.275268,0.107473,4.473739,16214.0
...,...,...,...,...,...,...,...,...,...,...,...
11691,0.643403,0.004187,-0.509601,-0.510820,0.931471,-0.251359,-0.654498,-0.448502,0.011662,-0.419596,120534.0
11692,0.392547,0.220755,-0.706443,-0.998427,-0.565426,-0.250086,-0.602604,-0.878397,0.047240,-0.367965,36036.0
11693,-5.958381,-0.875704,3.362062,0.320442,-1.232880,0.386524,-1.847455,3.821485,2.006521,0.735602,88416.0
